Billy's VAE

In [9]:
import os
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from DataMaker import *

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Reshape, Conv2D, Conv2DTranspose, Cropping2D, ZeroPadding2D, BatchNormalization, LeakyReLU
import tensorflow_probability as tfp
from tensorflow_probability import layers as tfpl
from tensorflow_probability import distributions as tfd
from tensorflow.nn import leaky_relu

In [11]:
DATASET_NAME = "HSC_v6_small"
MODEL_TYPE = "VAEB"
MODEL_VERSION = "v1"

model_id = '_'.join([DATASET_NAME, MODEL_TYPE, MODEL_VERSION])
dir_model = os.path.join('/models', model_id, 'model')
dir_checkpoints = os.path.join('/models', model_id, 'checkpoints')
dir_logs = os.path.join('/logs', model_id)
dir_predictions = os.path.join('/predictions', model_id)
weights_file = dir_model + '/weights.h5'

print(weights_file)
print(dir_model)
print(dir_checkpoints)
print(dir_logs)
print(dir_predictions)

os.makedirs(dir_model, exist_ok=True)
os.makedirs(dir_checkpoints, exist_ok=True)
os.makedirs(dir_logs, exist_ok=True)
os.makedirs(dir_predictions, exist_ok=True)

/models/HSC_v6_small_VAEB_v1/model/weights.h5
/models/HSC_v6_small_VAEB_v1/model
/models/HSC_v6_small_VAEB_v1/checkpoints
/logs/HSC_v6_small_VAEB_v1
/predictions/HSC_v6_small_VAEB_v1


In [12]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit= 10000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [13]:
hf_train = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_training_small.hdf5', 'r')
hf_test = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_testing_small.hdf5', 'r')
hf_validation = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_validation_small.hdf5', 'r')
x_train = np.asarray(hf_train['image'][0:])
x_test = np.asarray(hf_test['image'][0:])
x_validation = np.asarray(hf_validation['image'][0:])
max_value = 4.16
x_train = np.true_divide(x_train, max_value)
x_test = np.true_divide(x_test, max_value)
x_validation = np.true_divide(x_validation, max_value)
y_train = np.asarray(hf_train['specz_redshift'][0:])[..., None]
y_test = np.asarray(hf_test['specz_redshift'][0:])[..., None]
y_validation = np.asarray(hf_validation['specz_redshift'][0:])[..., None]
object_id_train = np.asarray(hf_train['object_id'][0:])
object_id = np.asarray(hf_test['object_id'][0:])
object_id_validation = np.asarray(hf_validation['object_id'][0:])
hf_train.close()
hf_test.close()
hf_validation.close()

In [10]:
INPUT_SHAPE = (5, 127, 127)

In [19]:
images = Input(shape=INPUT_SHAPE, name="encoder_input")
ec1 = Conv2D(filters = 32, kernel_size = (3, 3), padding = "same", activation = leaky_relu, strides = 1, data_format = 'channels_first')(images)
ebn1 = BatchNormalization(name="encoder_norm_1")(ec1)
ec2 = Conv2D(filters = 2, kernel_size = (3, 3), padding = "same", activation = leaky_relu, strides = 1, data_format = 'channels_first')(ebn1)
ebn2 = BatchNormalization(name="encoder_norm_1")(ec2)